# Make a pic
## Load table

In [ ]:
tfns = []
tfns += [('first',"pred_power_table.csv")]
#tfns += ['pred_power_table_nprefixes4_nlts3_perfs_XGB_CV1_red1.csv']
#tfns += ['pred_power_table_nprefixes4_nlts3_perfs_XGB_CV1_red0.csv']
tfns += [('Hisrch','pred_power_table_nprefixes4_nlts3_perfs_XGB_CV1_Hisrch.csv')]
#tfns += ['pptable_parcel_aal_nprefixes6_nlts1_perfs_XGB_CV1_red1.csv']
#tfns += ['pptable_parcel_aal_nprefixes6_nlts1_perfs_XGB_CV1_red0.csv']
tfns += [('full','pptable_parcel_aal_nprefixes9_nlts1_perfs_XGB_CV1_red1.csv')]
#tfns += ['pptable_parcel_aal_nprefixes9_nlts1_perfs_XGB_CV1_red0.csv']
# table_fnames_dict = {'red':table_fname1, 'full':table_fname2, 'Hirsch':table_fname3,
#                     'full2':table_fname4, 'red2':table_fname5, 'full3':table_fname6,
#                     'red3':table_fname7}
#table_labels = list(sorted(table_fnames_dict.keys() ))
#table_labels = list((table_fnames_dict.keys() ))

table_labels = [tfn[0] for tfn in tfns]
#table_labels = ['Hirsch']

In [ ]:
table_fname = tfns[0][1]
df = pd.read_csv(table_fname)
num_featsets = len(df.keys())
#for tfn in table_labels:
print(num_featsets)

In [ ]:
#seq = np.linspace(0,1,num_featsets-2)
seq = np.random.permutation(num_featsets).astype(float); seq /= np.max(seq)
cmap = plt.cm.gist_rainbow
#cmap = plt.cm.hsv
cyc = plt.cycler("color", cmap(seq ))

plt.cm.gist_rainbow

# for ci,c in enumerate(cyc):
#     print(c)

In [ ]:
des_label = 'allsep'
des_featsets = ['all', 'allb_beta', 'modLFP', 'modSrc']
featset_dispname = ['MEG+LFP', 'MEG beta+LFP beta', 'LFP', 'MEG']
table_dispname = ['Entire hemisphere', '4 sources', 'Entire hemisphere2']

In [ ]:
ww = 7
hh = ww
nr = len(table_labels)
nc = 2
fig,axs = plt.subplots(nr,nc, figsize=(ww*2,hh*nr))
axs = axs.reshape((nr,nc))

for ax in axs.flatten():
    ax.set_xlim(-3,103)
    ax.set_ylim(-3,103)
    ax.set_ylim(40,103)
    ax.set_xlabel('Sensitivity',fontsize=22)
    ax.set_ylabel('Specificity',fontsize=22)

for rowi in range(nr):
    table_lbl_cur = table_labels[rowi]
    #tfn = table_fnames_dict[table_lbl_cur]
    tfn = tfns[rowi][1]
    df = pd.read_csv(tfn)
    
    ax  = axs[rowi,0]
    ax1 = axs[rowi,1]
    defsz = 20

    larger= ['all', 'modLFP', 'modSrc', 'allb_beta']

    num_featsets_cur = len(df.keys())
    featsetcolori = 0
    for ind in range(num_featsets_cur): #range(1,num_featsets-1):
    #ind = 1
        indd = 0
        try:
            indd = int(df.keys()[ind])  #column
        except (ValueError,KeyError) as e:
            indd = 0
        
        if indd <= 0:
            continue
        cc  = list(cyc)[featsetcolori]
        key = str(indd)
        featset_name = df[key][0]
        if featset_name not in des_featsets:
            continue
            
        featsetcolori += 1
        print('{}: featset name {}'.format(table_lbl_cur,featset_name) )
        
        l = len(df[key])
        l = 0
        nice_inds = []
        for i in range(1,len(df[key])):
            if df['0'][i].find(des_label) >= 0:
                #print(df['0'][i])
                l += 1
                nice_inds += [i]

        # specs over patients
#         senss = (l-1)*[0]
#         specs = (l-1)*[0]
        senss = (l)*[0]
        specs = (l)*[0]
        #print(len(specs),nice_inds)
        #for i in range(1,l):
        for ii,i in enumerate(nice_inds):
            s = df[key][i]
            try:
                r = list(map(int,s.split(',')) )
            except ValueError as ve:
                continue
            se = r[0]
            sp = r[1]
            if len(r) > 2:
                f1 =  r[2]
            #print(i)
            senss[ii] = se
            specs[ii] = sp
            #print( se,sp  )
        #print('fd0')
                
        clrs = len(specs) *  [ cc['color'] ]
        sz = defsz
        mrk = 'o'
        if featset_name in larger:
            sz *= 8        
            mrk = 'o'
        ase = np.mean(senss)
        asp = np.mean(specs)
        lbl = '{} ~ {:.0f},{:.0f}'.format(featset_name, ase, asp)
        lbl = featset_dispname[des_featsets.index(featset_name)]
        
        #print('fd1')
        ax.scatter(senss,specs, c=clrs , label=lbl, s= sz, marker=mrk, alpha=0.8)

        ax1.scatter([ase],[asp], c=[clrs[0]] , label=lbl, s= sz, marker=mrk)

    ax.legend(loc='lower left',fontsize=20); 
    ttl = table_lbl_cur
    ttl = table_dispname[rowi]
    ax.set_title('{}, across patients'.format(ttl),
                                              fontsize=25)
    ax1.legend(loc='lower left'); ax1.set_title('{} Average predictive powers'.format(table_lbl_cur),
                                                fontsize=15)
    ax.grid(1)
    ax1.grid(1)

plt.savefig('pred_power_scatter_o{}.pdf'.format(table_fname))